In [4]:
from typing import NamedTuple, List
from cs103 import *

## School Tuition

Given a list of schools across the world, a location of residence (the
place where you live), and an alternate
location you're considering, find the school with the lowest tuition
in one of the two locations under consideration. 

Assume that the list of schools you are being given is not going to be an empty list and that the list contains at least one school from the residence you are interested in.

In [5]:
School = NamedTuple('School', [('name', str),
                               ('location', str),
                               ('local_tuition', int),       # in range[0,...]
                               ('non_local_tuition', int)])  # in range[0,...]
# interp. Schools with their name, location (as a string, which may be a province or
# other location), local tuition, and non-local tuition.
S1 = School('School_1', 'Canada', 100, 1000)
S2 = School('School_2', 'China', 50, 800)
S3 = School('School_3', 'USA', 400, 5000)
S4 = School('School_4', 'Australia', 30, 300)
S5 = School('School_5', 'Hong Kong', 250, 2500)
S6 = School('School_6', 'Brazil', 500, 500)
S7 = School('School_7', 'Canada', 90, 400)

# template based on compound (4 fields)
@typecheck
def fn_for_school(s: School) -> ...:
    return ...(s.name,
               s.location,
               s.local_tuition,
               s.non_local_tuition)



# List[School]
# interp. a list of schools

L1 = []
L2 = [S1, S2]
L3 = [S1, S2, S3, S4]
L4 = [S1, S2, S3, S4, S5, S6, S7]

# template based on arbitrary-sized with the reference rule
@typecheck
def fn_for_los(los: List[School]) -> ...:
    # description of acc
    acc = ... # type: ...
    
    for s in los:
        acc = ...(fn_for_school(s), acc)
        
    return ...(acc)

In [9]:
# We've already got a signature, purpose, stub, and examples.
# Let's start by writing out a plan of what we want to do in English.
# (If the plan has multiple, separate steps, then we will NOT template based
# on List[School] but rather based on function composition!)

@typecheck
def find_lowest_tuition_in_areas(los: List[School], residence: str, alternate: str) -> School:
    """
    Given los (which must contain at least one school located in either residence or alternate),
    produce the school located in either residence or alternate that has the lowest tuition,
    given that we reside in the location residence (which affects local vs. non-local tuition
    for schools). In case of ties, produce the school in the alternate location.
    """
    #return S1  # stub
    # template based on function composition
    return lowest_tuition(filter_by_location(los, residence, alternate), 
                          residence, 
                          alternate)

@typecheck 
def filter_by_location(los: List[School], residence: str, alternate: str) -> List[School]:
    """
    return a list of schools in los that are in either residence or alternate location
    """
    #return los
    # template from List[School] with two additional parameters
    # acc stores the result so far
    acc = [] # type: List[School]
    
    for s in los:
        if in_either_location(s, residence, alternate):
            acc.append(s)
        
    return acc 

@typecheck
def in_either_location(s: School, residence: str, alternate: str) -> bool:
    """
    return True if s is in location residence or alternate, False otherwise
    """
    #return True
    #template from School with two additional parameters
    return s.location == residence or s.location == alternate

@typecheck 
def lowest_tuition(los: List[School], residence: str, alternate: str) -> School:
    """
    return the school that has the lower tuition between the local 
    tuition for location residence or the non-local tuition for
    location alternate
    CONSTRAINT: los is not empty and contains only schools in locations
    residence or alternate
    """
    #return S1
    # template from List[School]
    # acc contains the result so far
    acc = los[0] # type: School
    
    for s in los:
        if choose_school(s, acc, residence, alternate):
            acc = s
        
    return acc

@typecheck
def choose_school(s: School, curr: School, res: str, alt: str) -> bool:
    """
    return True if s should be chosen (s has cheaper tuition than curr, given the
    residence and alternate locations or the tuition is the same and s
    is in the alternate location)
    """
    #return True
    # template from School (twice) with two additional parameters
    s_tuition = find_tuition(s, res)
    curr_tuition = find_tuition(curr, res)
        
    if s_tuition == curr_tuition:
        return s == choose_alternate(s, curr, res, alt)

    return s_tuition < curr_tuition

@typecheck
def find_tuition(s: School, res: str) -> int:
    """
    return the local or non-local tuition depending on res
    """
    if s.location == res:
        return s.local_tuition
    else:
        return s.non_local_tuition
    
@typecheck
def choose_alternate(s1: School, s2: School, res: str, alt: str) -> School:
    """
    to break tuition ties, return the School that is in the alt location
    if both are in the alt location, return s1
    """
    if s1.location == alt and s2.location == alt:
        return s1
    elif s2.location == alt:
        return s2
    else:
        return s1



start_testing()

expect(find_lowest_tuition_in_areas(L2, "Canada", "Australia"), S1)
expect(find_lowest_tuition_in_areas(L4, "Canada", "Australia"), S7)
expect(find_lowest_tuition_in_areas(L4, "USA", "Australia"), S4)
expect(find_lowest_tuition_in_areas([S3, S7], "USA", "Canada"), S7)

expect(filter_by_location([], "Canada", "China"), [])
expect(filter_by_location(L2, "Canada", "China"), L2)
expect(filter_by_location(L2, "China", "Canada"), L2)
expect(filter_by_location(L2, "USA", "Mexico"), [])
expect(filter_by_location(L2, "Canada", "England"), [S1])
expect(filter_by_location(L2, "China", "Thailand"), [S2])
expect(filter_by_location(L4, "Brazil", "Canada"), [S1, S6, S7])

expect(in_either_location(S1, "Canada", "China"), True)
expect(in_either_location(S1, "China", "Canada"), True)
expect(in_either_location(S1, "Brazil", "Mexico"), False)
expect(in_either_location(S6, "Canada", "China"), False)
expect(in_either_location(S6, "China", "Brazil"), True)
expect(in_either_location(S6, "Brazil", "Mexico"), True)

expect(lowest_tuition(L2, "Canada", "China"), S1)
expect(lowest_tuition(L2, "China", "Canada"), S2)
expect(lowest_tuition([S3, S6], "Brazil", "USA"), S6)
expect(lowest_tuition([S1, S7, S4], "Canada", "Australia"), S7)

expect(choose_school(S1, S7, "Canada", "Australia"), False)
expect(choose_school(S7, S1, "Canada", "Australia"), True)
expect(choose_school(S1, S2, "Canada", "China"), True)
expect(choose_school(S2, S1, "Canada", "China"), False)
expect(choose_school(S1, S2, "China", "Canada"), False)
expect(choose_school(S2, S1, "China", "Canada"), True)

expect(find_tuition(S1, "Canada"), 100)
expect(find_tuition(S1, "USA"), 1000)
expect(find_tuition(S2, "China"), 50)
expect(find_tuition(S2, "Australia"), 800)

expect(choose_alternate(S1, 
                        School('School_8', 'USA', 100, 100), 
                        "Canada", 
                        "USA"), 
       School('School_8', 'USA', 100, 100))
expect(choose_alternate(S3, S7, "USA", "Canada"), S7)

summary()


33 of 33 tests passed


## Quarterly Stock Volumes

**Problem:** Find the average volume traded of a stock in a given quarter (Jan-Mar, Apr-Jun, Jul-Sep, Oct-Dec).

There's code below already, and it's correct, but its collection of three accumulators is a bit messy. Refactor it to use composition so it's clearer!

Note that `len` is a function that works on lists of any type as well as strings.

**We may not discuss this in class. Be sure to check out the sample solution when it releases!**

In [ ]:
from typing import List

QuarterlyVolumes = List[int] # in range [0, ...]
# interp. the quarterly volumes traded of a stock (i.e., amounts for
# quarter 1 (Jan-Mar), 2 (Apr-Jun), 3 (Jul-Sep), or 4 (Oct-Dec).
# Examples should be in contiguous time order. In other words, they
# should not skip quarters!
LOQV0 = []
LOQV_ONE_YEAR = [0, 100, 25, 1]
LOQV_THREE_YEARS = [0, 100, 25, 1, 
                    10, 50, 14, 3, 
                    80, 22, 17, 100]
LOQV_AAPL_1981_2017 = [
    407831200, 519478400, 489932800, 631993600,
    668074400, 949284000, 1309442400, 2414451200,
    2595163200, 2057680800, 3525244800, 2950164000,
    2359016800, 2926879200, 2611828800, 2597033600,
    3928848000, 3329636800, 1554384800, 2560448800,
    3494248800, 3370024000, 3228338400, 3238194400,
    5211102400, 3537413600, 2316260800, 3878050400,
    3618042400, 2367792000, 1975013600, 2362396400,
    4022944800, 3290271600, 2265446400, 3147793600,
    2696965600, 2927540000, 2374775200, 3101204400,
    4448421600, 4826889200, 2587303600, 2474298400,
    2555954800, 2871699600, 2358750800, 2498073200,
    3463768000, 3697338400, 3623524800, 3328600800,
    4218620000, 3018895600, 3163902000, 3887556400,
    5114648000, 4395230000, 4575995200, 4480761600,
    4574766000, 2786229600, 2801103200, 3136456400,
    4537288000, 2168684000, 7016038400, 4268829600,
    7047930400, 4698766800, 8057212800, 8994638800,
    8298978800, 7929124000, 9919663600, 8127910000,
    7143329200, 6643840000, 6474946800, 9813283200,
    7883072400, 6566422800, 4828996200, 4385957800,
    5417846000, 5276752600, 4640484800, 3918398400,
    3547135200, 6175055600, 3718330000, 4367042400,
    6118987000, 5615421000, 6345362800, 12370646400,
    15458277800, 10890252100, 7960705900, 11291009800,
    15957925200, 13416171300, 13272236600, 11278408400,
    14212695000, 13526087400, 17327648800, 16682565200,
    20741779100, 15212747900, 14225769600, 21315004900,
    11849447400, 8488599000, 7274145200, 8201230100,
    9524811800, 11775259300, 9277186100, 7178974600,
    7860986000, 6357435000, 9693498500, 7102915400,
    8453186700, 8640290400, 6596215500, 9301358500,
    7911083600, 6857389000, 5806848600, 5030071200,
    4912005000, 4249617100, 3498583000, 3254283000,
    3579821100, 2828800200, 4050072500, 2784493400,
    2827936300, 2551770000, 2283923700, 2016991500,
    1699719000, 1711464000, 1763455000, 1635788200
]

@typecheck
# template based on arbitrary-sized
def fn_for_loqv(loqv: List[int]) -> ...:
    # description of the accumulator
    acc = ...   # type: ...

    for qv in loqv:
        acc = ...(qv, acc)

    return ...(acc)

In [ ]:
@typecheck
def avg_volume_of_quarter(loqv: QuarterlyVolumes, quarter: int) -> float:
    """
    Return the average volume for the given stock in the given quarter across all years of data supplied.
    
    Assumes that loqv begins with quarter 1. quarter must be in the range [1,4].
    
    Returns 0 if there are no matching quarters.
    """
    #return 0.0  #stub
    #template based on QuarterlyVolumes
    
    # total is the total volume across quarters matching quarter seen so far
    total = 0   # type: int
    
    # count is the number of quarters matching quarter seen so far
    count = 0   # type: int
    
    # curr is the current quarter for the quarter currently under review
    curr = 1    # type: int  # in range[1,4]
    
    for qv in loqv:
        if quarter == curr:
            total += qv
            count += 1
        curr += 1
        if curr > 4:
            curr = 1
    
    if count == 0:
        return 0.0
    else:
        return total / count


start_testing()

# An extra example that has an incomplete year:
LOQV_INCOMPLETE_YEAR = [0, 100, 25, 1, 
                        10, 50, 14, 3, 
                        80, 22]

expect(avg_volume_of_quarter([], 1), 0.0)
expect(avg_volume_of_quarter([], 3), 0.0)
expect(avg_volume_of_quarter([10, 20, 300, 4000], 1), 10.0)
expect(avg_volume_of_quarter([10, 20, 300, 4000], 2), 20.0)
expect(avg_volume_of_quarter([10, 20, 300, 4000], 3), 300.0)
expect(avg_volume_of_quarter([10, 20, 300, 4000], 4), 4000.0)

expect(avg_volume_of_quarter(LOQV_THREE_YEARS, 1), (0+10+80)/3)
expect(avg_volume_of_quarter(LOQV_THREE_YEARS, 3), (25+14+17)/3)

expect(avg_volume_of_quarter(LOQV_INCOMPLETE_YEAR, 1), (0+10+80)/3)
expect(avg_volume_of_quarter(LOQV_INCOMPLETE_YEAR, 3), (25+14)/2)

summary()